## Setup

In [4]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

C:\Users\dlr93\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


## Creating the agents

In [5]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [6]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [7]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [8]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

In [9]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [10]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


C:\Users\dlr93\anaconda3\Lib\site-packages\autogen\agentchat\chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Daniel
customer_proxy_agent (to Onboarding Personal Information Agent):

Daniel

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing your name. Could you also tell me your location?

--------------------------------------------------------------------------------
Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Mexico
customer_proxy_agent (to Onboarding Personal Information Agent):

Mexico

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

**********************************************************************

## Summary and cost

In [11]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Daniel",
  "location": "Mexico"
}


Daniel from Mexico is interested in reading about Soccer.


Daniel from Mexico is interested in reading about soccer. Mexico has a strong soccer culture and has participated in the FIFA World Cup since 1930. There have been unforgettable moments in soccer history, and he can explore more about the exciting world of soccer.




In [12]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0001365, 'gpt-3.5-turbo-0125': {'cost': 0.0001365, 'prompt_tokens': 180, 'completion_tokens': 31, 'total_tokens': 211}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001365, 'gpt-3.5-turbo-0125': {'cost': 0.0001365, 'prompt_tokens': 180, 'completion_tokens': 31, 'total_tokens': 211}}}


{'usage_including_cached_inference': {'total_cost': 4.85e-05, 'gpt-3.5-turbo-0125': {'cost': 4.85e-05, 'prompt_tokens': 67, 'completion_tokens': 10, 'total_tokens': 77}}, 'usage_excluding_cached_inference': {'total_cost': 4.85e-05, 'gpt-3.5-turbo-0125': {'cost': 4.85e-05, 'prompt_tokens': 67, 'completion_tokens': 10, 'total_tokens': 77}}}


{'usage_including_cached_inference': {'total_cost': 0.000499, 'gpt-3.5-turbo-0125': {'cost': 0.000499, 'prompt_tokens': 347, 'completion_tokens': 217, 'total_tokens': 564}}, 'usage_excluding_cached_inference': {'total_cost': 0.000499, 'gpt-3.5-turbo-0125': {'cost': 0.000499, 'prompt_tokens': 347, 'complet